In [1]:
import pandas as pd  # used for data manipulation
import csv           # used for working with csv files
from bs4 import BeautifulSoup  # web scraping
from requests import get
from requests.exceptions import RequestException   # add comments for import statements
from contextlib import closing


# Disable warnings in Anaconda
import warnings
warnings.simplefilter('ignore')




In [2]:
# Code Reference: https://realpython.com/python-web-scraping-practical-introduction/

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200
            and content_type is not None
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors.
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [3]:
sector_data = pd.DataFrame(columns=['Name','Address','Sector'])


In [4]:

for i in range(1, 538):
#     if i%50 == 0:
#         print(i)
    url = 'https://gender-pay-gap.service.gov.uk/viewing/search-results?p={}'.format(i)
    raw_html = simple_get(url)
    html = BeautifulSoup(raw_html, 'html.parser')

    sectors = html.findAll("dd", {"class": "metadata-text-value"})
    sector = [sector.text.strip() for sector in sectors]

    addresses = html.findAll("address", {"class": ""})
    address = [address.text.strip() for address in addresses]

    names = html.findAll("a", {"data-name": "ViewLink"})
    name = [name.text.strip() for name in names]

    d = {'Name': name, 'Address': address, 'Sector' : sector}
    df = pd.DataFrame(data=d)

    sector_data = sector_data.append(df)




In [5]:
sector_data['Sector']  = sector_data['Sector'].str.replace('\r\n',"")

In [6]:
dataset = pd.read_csv('/Users/virteep/IndependentStudy/UK Gender Pay Gap Data - 2017 to 2018.csv')

In [7]:
dataset['DateSubmitted'] = pd.to_datetime(dataset['DateSubmitted'])
dataset['DueDate'] = pd.to_datetime(dataset['DueDate'])

In [8]:
dataset['EmployerSize'] = dataset['EmployerSize'].astype('category')

In [9]:
dataset['SicCodes']  = dataset['SicCodes'].str.replace('\r\n',"")

In [10]:
dataset.drop(columns='Address', index=1, inplace=True)

In [11]:
data_sector = dataset.merge(sector_data, left_on = 'CurrentName', right_on='Name', how = 'left')

In [12]:
#Ref: https://www.civilsociety.co.uk/voices/david-kane-an-analysis-of-the-gender-pay-gap-in-charities.html
charity_data = pd.read_csv('gender-pay-gap-with-charity-no.csv', encoding='latin-1')

In [13]:
charity_data = charity_data[['CurrentName','OrgType', 'OrgSubType', 'PaidMore', 'FemaleWorkforce']]

In [14]:
final_data = data_sector.merge(charity_data, on = 'CurrentName', how = 'left')

In [ ]:

# DiffMeanHourlyPercent       Their mean gender pay gap
# DiffMedianHourlyPercent     Their median gender pay gap
# DiffMeanBonusPercent        Their mean bonus gender pay gap
# DiffMedianBonusPercent      Their median bonus gender pay gap
# MaleBonusPercent            The proportion of men in the organisation receiving a bonus payment
# FemaleBonusPercent          The proportion of women the organisation receiving a bonus payment
# MaleLowerQuartile, FemaleLowerQuartile                 
# MaleLowerMiddleQuartile, FemaleLowerMiddleQuartile           
# MaleUpperMiddleQuartile, FemaleUpperMiddleQuartile           
# MaleTopQuartile, FemaleTopQuartile                
#                                      The proportion of men and women in each quartile pay band

In [15]:
final_data['Postcode'] = final_data['Address'].str[-8:]
final_data['Postcode'] = final_data['Postcode'].str.replace(",","")
final_data['Postcode'] = final_data['Postcode'].str.strip()

In [16]:
postcodes = pd.read_csv('postcodes.csv', usecols=['Postcode', 'Latitude', 'Longitude'])

In [17]:
data_with_lat_lon = final_data.merge(postcodes, how = 'inner', on='Postcode')

In [18]:
neutral = (data_with_lat_lon['DiffMedianHourlyPercent'] <= 5) & (data_with_lat_lon['DiffMedianHourlyPercent'] >= -5)
women_paid_more = data_with_lat_lon['DiffMedianHourlyPercent'] < -5
men_paid_more = data_with_lat_lon['DiffMedianHourlyPercent'] > 5

data_with_lat_lon.loc[neutral, 'Target'] = 'EqualPay'
data_with_lat_lon.loc[women_paid_more, 'Target'] = 'Women'
data_with_lat_lon.loc[men_paid_more, 'Target'] = 'Men'

In [21]:
data_with_lat_lon.to_csv('UK_gpg_data.csv',index=False)